# OpenStreetMap Case Study

In [ ]:
from street_suffix import map_street_suffix
from audit import audit_addresses
from update import update_addr
from prepare_for_database import process_map

## Map Area

#### Roswell, GA, US 

- http://www.openstreetmap.org/relation/119569

I moved to this area 2 years ago, and chose to use this map for my project as a way to learn more about the area and improve the OpenStreetMap data

In [ ]:
osm_file = 'roswell.osm'

## Problems Encountered in the Map

In [ ]:
expected, mapping = map_street_suffix()

In [ ]:
audit_addresses(osm_file, expected)

### 1. Inconsistent address abbreviations
('Glen Meadows Dr NW','Clubland Drive Northeast'

To clean this field, I scraped the https://pe.usps.com/text/pub28/28apc_002.htm website to get a listing of all expected street suffix abbreviations mapped to the full street suffix name.  As well, I added mappings for cardinal directions (North, South, East, West, etc) and rather than using regular expressions to update the last word in the street name, I split the street name and iterated through each word (ex. Glen Meadows Dr NW => Glen Meadows Drive Northwest rather than Glen Meadows Dr Northwest)
    

In [ ]:
def update_addr(key, value, mapping, expected):
    if key == "street":
            value_split = value.split()
            i = 0
            new_value = ""
            while i < len(value_split):
                if value_split[i] != None:
                    word = value_split[i]
                    word = word.capitalize().replace(".", "")
                    word = update_word(word, mapping, expected)
                    new_value += word + " "
                i += 1
            return new_value

### 2. Misspelled city, unexpected city and state and zip in the city field
('Sandy Springa', 'Bismarck', 'GA 30350')

I corrected the misspelling of Sandy Springs programmatically. Upon further investigation of 'Bismarck's' node_tag ID, this value is associated with Ridey Taxi Service in Bismarck, ND 58502.  Since this information does not belong in this dataset, I chose to delete it manually from the database. Investigation into the 'GA 30350' value showed that for id # 42882100 the city and postcode fields were switched.  I chose to manually update this in the database as well.

In [ ]:
    elif key == "city":
            if value == "Sandy Springa":
                value = "Sandy Springs"
            return value

### 3. Inconsistent State abbreviations and capitalization, as well as inaccurate state's
('GA', 'Georgia', 'ND', 'ga')

I Chose to maintain a capitalized abbreviation (GA) and updated all fields to match.  The unexpected 'ND' value was also taken care of in the deletion explained above.

In [ ]:
    elif key == "state":
            if value == "Georgia":
                value = "GA"
            return value.upper()

### 4. Inconsistent and incorrect zip codes, as well as city names in the zip code field
('58502', 'Atlanta,', '30092-4207', '1879')

The unexpected '58502' value was taken care of in the deletion explained above.  I chose to standardize to just the 5 digit zip code.  '1879' belongs to the id associated with 'Burger King # 4089' in Norcross, GA so the zip code should actually be '30092'.  This was easily corrected in the database.  The value 'Atlanta' was taken care of in the manual update to id # 42882100 described above

In [ ]:
    elif key == "postcode":
            return value[0:5]
        else:
            return value

In [ ]:
process_map(osm_file, validate=False)

## Upload to SQL Database

In [ ]:
from create_database import import_table
import sqlite3
import csv
from pprint import pprint

connection = sqlite3.connect('roswell.db')
cursor = connection.cursor()
import_table('roswell.db', 'csv_files/nodes_tags.csv', 'nodes_tags', connection, cursor)
import_table('roswell.db', 'csv_files/nodes.csv', 'nodes', connection, cursor)
import_table('roswell.db', 'csv_files/ways_nodes.csv', 'ways_nodes', connection, cursor)
import_table('roswell.db', 'csv_files/ways_tags.csv', 'ways_tags', connection, cursor)
import_table('roswell.db', 'csv_files/ways.csv', 'ways', connection, cursor)